In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from UNet3D import UNet3D

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create the UNet3D model
unet3d_model = UNet3D(
    in_channels=1,
    out_channels=1,
    base_channels=32,
    layers_per_block=1,
    block_out_channels=(64, 128, 256, 512),
    time_embed_dim=256,
    attention_head_dim=8
)

Using device: cpu


In [8]:
from diffusion import DiffusionModel3D

diffusion_model = DiffusionModel3D(unet3d_model, num_timesteps=1000)
diffusion_model.to(device)

# Print model summary
print(diffusion_model)
print(f"Total parameters: {sum(p.numel() for p in diffusion_model.parameters())}")

DiffusionModel3D(
  (model): UNet3D(
    (conv_in): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (time_embedding): TimestepEmbedding(
      (linear_1): Linear(in_features=1, out_features=256, bias=True)
      (activation): SiLU()
      (linear_2): Linear(in_features=256, out_features=256, bias=True)
    )
    (down_blocks): ModuleList(
      (0): DownBlock3D(
        (resnets): ModuleList(
          (0): ResnetBlock3D(
            (norm1): GroupNorm(32, 32, eps=1e-05, affine=True)
            (conv1): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (norm2): GroupNorm(32, 64, eps=1e-05, affine=True)
            (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
            (skip): Conv3d(32, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1))
            (time_emb_proj): Linear(in_features=256, out_features=64, bias=True)
          )
        )
        (downsample): Conv3d(64, 64, kernel_siz

In [ ]:
from training import train_diffusion_model
from data_loader import create_dataloader

# Reduce batch size if needed
batch_size = 2  # or even 1 if still having memory issues
data_dir = "./objaverse_processed/processed_models"
train_dataloader, test_dataloader = create_dataloader(data_dir, batch_size=batch_size)

print(f"Train dataset size: {len(train_dataloader.dataset)}")
print(f"Test dataset size: {len(test_dataloader.dataset)}")

# Train the model
try:
    losses, test_losses = train_diffusion_model(diffusion_model, train_dataloader, test_dataloader, epochs=30, device=device)
except Exception as e:
    print(f"An error occurred during training: {e}")
    raise

In [ ]:
# Plot the training loss
import matplotlib.pyplot as plt
plt.plot(losses)
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


In [14]:
from inference import DiffusionInference3D

# Perform inference and generate new samples
inference = DiffusionInference3D(diffusion_model, diffusion_model.noise_scheduler, device=device)
samples = inference.sample(num_samples=4, image_size=image_size)

# Visualize the generated samples
inference.visualize_samples(samples)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
